# Introduction to CUDA parallel programming

## Setting up the environment

### Load nvcc plugin

In [ ]:
%load_ext nvcc4jupyter

### For older GPU architectures (e.g. Maxwell) set additional command line parameters

In [2]:
MAXWELL_ARCH_CMD = "--gpu-architecture=compute_50 --gpu-code=compute_50,sm_50,sm_52"

## RGB2Gray

In [3]:
%%writefile ppmIO.h

#pragma once
#include <fstream>


inline void getPPMSize(const char *filename, unsigned int* width, unsigned int* height) {
	char buff[16];
	FILE *fp;
	int c;

	//open PPM file for reading
	fp = fopen(filename, "rb");
	if (!fp) {
	  fprintf(stderr, "Unable to open file '%s'\n", filename);
	  exit(1);
	}

	//read image format
	if (!fgets(buff, sizeof(buff), fp)) {
	  perror(filename);
	  exit(1);
	}

	//check the image format
	if (buff[0] != 'P' || buff[1] != '6') {
		 fprintf(stderr, "Invalid image format (must be 'P6')\n");
		 exit(1);
	}

	//check for comments
	c = getc(fp);
	while (c == '#') {
	while (getc(fp) != '\n') ;
		 c = getc(fp);
	}

	ungetc(c, fp);
	//read image size information
	if (fscanf(fp, "%d %d", &(*width), &(*height)) != 2) {
		 fprintf(stderr, "Invalid image size (error loading '%s')\n", filename);
		 exit(1);
	}
}

inline void readPPM(const char *filename, float* image)
{
	 char buff[16];
	 FILE *fp;
	 int c, rgb_comp_color;
	 unsigned int width, height;

	 //open PPM file for reading
	 fp = fopen(filename, "rb");
	 if (!fp) {
		  fprintf(stderr, "Unable to open file '%s'\n", filename);
		  exit(1);
	 }

	 //read image format
	 if (!fgets(buff, sizeof(buff), fp)) {
		  perror(filename);
		  exit(1);
	 }

	//check the image format
	if (buff[0] != 'P' || buff[1] != '6') {
		 fprintf(stderr, "Invalid image format (must be 'P6')\n");
		 exit(1);
	}

	//check for comments
	c = getc(fp);
	while (c == '#') {
	while (getc(fp) != '\n') ;
		 c = getc(fp);
	}

	ungetc(c, fp);
	//read image size information
	if (fscanf(fp, "%d %d", &width, &height) != 2) {
		 fprintf(stderr, "Invalid image size (error loading '%s')\n", filename);
		 exit(1);
	}

	//read rgb component
	if (fscanf(fp, "%d", &rgb_comp_color) != 1) {
		 fprintf(stderr, "Invalid rgb component (error loading '%s')\n", filename);
		 exit(1);
	}

	//check rgb component depth
	if (rgb_comp_color!= 255) {
		 fprintf(stderr, "'%s' does not have 8-bits components\n", filename);
		 exit(1);
	}

	while (fgetc(fp) != '\n') ;

	//allocate temporary memory for unsigned char data
	unsigned char* tempImage = (unsigned char*) malloc (width*height*3*sizeof(unsigned char));

	//read pixel data from file
	if (fread(tempImage, sizeof(unsigned char), 3 * width * height, fp) != 3 * width * height) {
		 fprintf(stderr, "Error loading image '%s'\n", filename);
		 exit(1);
	}

	// transfer data from unsigned char to float
	int i;
	for (i = 0; i < width * height * 3; ++i) {
		image[i] = (float)tempImage[i];
	}

	//cleanup
	free(tempImage);
	fclose(fp);
}

void writePPM(const char* filename, float* image, unsigned int width, unsigned int height, bool isGray=0)
{
	FILE *fp;
	unsigned int channels = isGray ? 1 : 3;
	//open file for output
	fp = fopen(filename, "wb");
	if (!fp) {
		 fprintf(stderr, "Unable to open file '%s'\n", filename);
		 exit(1);
	}

	//write the header file
	//image format
	fprintf(fp, "P6\n");

	//comments
	fprintf(fp, "# Created for CUDA labs AGH\n");

	//image size
	fprintf(fp, "%d %d\n", width, height);

	// rgb component depth
	fprintf(fp, "%d\n", 255);

	//copy from float* to uchar*
	unsigned char* tempImage = (unsigned char*) malloc (width*height*3*sizeof(unsigned char));
	int i, j=0;
		for (i = 0; i < width * height * channels; ++i) {

			j = isGray ? j : i;
			tempImage[j] = (unsigned char)image[i];

			if (isGray) {
				tempImage[j+1] = (unsigned char)image[i];
				tempImage[j+2] = (unsigned char)image[i];
				j += 3;
			}
		}

	// pixel data
	fwrite(tempImage, sizeof(unsigned char), 3 * width * height, fp);
	free(tempImage);
	fclose(fp);
}

Overwriting ppmIO.h


In [4]:
%%writefile "RGB2Gray/rgb2gray.cu"

#include "../ppmIO.h"
#include <stdio.h>


#define TILE_WIDTH 16
__global__ void rgb2gray(float *grayImage, float *rgbImage, int channels, int width, int height) {
  int x = threadIdx.x + blockIdx.x * blockDim.x;
  int y = threadIdx.y + blockIdx.y * blockDim.y;

  if (x < width && y < height) {
    // get 1D coordinate for the grayscale image
    int grayOffset = y * width + x;
    // one can think of the RGB image having
    // CHANNEL times columns than the gray scale image
    int rgbOffset = grayOffset * channels;
    float r       = rgbImage[rgbOffset];     // red value for pixel
    float g       = rgbImage[rgbOffset + 1]; // green value for pixel
    float b       = rgbImage[rgbOffset + 2]; // blue value for pixel
    // perform the rescaling and store it
    // We multiply by floating point constants
    grayImage[grayOffset] = 0.21f * r + 0.71f * g + 0.07f * b;
  }
}


int main(int argc, char *argv[]) {

  // check if number of input args is correct: input and output image filename
  if (argc != 3) {
	  printf("Wrong number of arguments: exactly 2 arguments needed (input and output .ppm filename)\n");
	  return 0;
  }

  // get size of input image
  unsigned int width, height;
  getPPMSize(argv[1], &width, &height);
  // read input image to a host variable
  float* hostInputImageData = (float*) malloc (width*height*3*sizeof(float));
  readPPM(argv[1], hostInputImageData);

  // allocate input and output images in the device
  float *deviceInputImageData;
  float *deviceOutputImageData;
  cudaMalloc((void **)&deviceInputImageData, width * height * 3 * sizeof(float));
  cudaMalloc((void **)&deviceOutputImageData, width * height * sizeof(float));

  // copy image to the device
  cudaMemcpy(deviceInputImageData, hostInputImageData, width * height * 3 * sizeof(float), cudaMemcpyHostToDevice);

  dim3 dimGrid(ceil((float)width / TILE_WIDTH),
               ceil((float)height / TILE_WIDTH));
  dim3 dimBlock(TILE_WIDTH, TILE_WIDTH, 1);
  rgb2gray<<<dimGrid, dimBlock>>>(deviceOutputImageData, deviceInputImageData, 3, width, height);

  float *hostOutputImageData = (float*) malloc (width*height*sizeof(float));
  cudaMemcpy(hostOutputImageData, deviceOutputImageData, width * height * sizeof(float), cudaMemcpyDeviceToHost);
  writePPM(argv[2], hostOutputImageData, width, height, 1);

  free(hostInputImageData);
  free(hostOutputImageData);
  cudaFree(deviceInputImageData);
  cudaFree(deviceOutputImageData);

  return 0;
}

Overwriting RGB2Gray/rgb2gray.cu


### Compile

In [ ]:
!nvcc "RGB2Gray/rgb2gray.cu" -o "RGB2Gray/a.out" $MAXWELL_ARCH_CMD

### Run

In [6]:
!"RGB2Gray/a.out" "RGB2Gray/dog_512.ppm" "RGB2Gray/output.ppm"

### Verify

In [7]:
!compute-sanitizer --tool memcheck "RGB2Gray/a.out" "RGB2Gray/dog_512.ppm" "RGB2Gray/output.ppm"

========= COMPUTE-SANITIZER
========= ERROR SUMMARY: 0 errors


## Matrix Multiplication

In [8]:
%%writefile "MMul/mmul.cu"

#include <stdio.h>
#include <cuda.h>


__global__ void multMatrixVector(float* B, float* C, float* A, unsigned int size) {
  int x = threadIdx.x + blockIdx.x * blockDim.x;
  float sum = 0.0f;
  if (x < size)
  {
    for (int i = 0; i < size; i++)
    {
      sum += B[x*size+i] * C[i];
    }
  A[x] = sum;
  }
}
//

int main(int argc, char **argv)
{
	float h_B[] = {0, 1, 2, 3, 4, 5, 6, 7, 8};
	float h_C[] = {1, 2, 3};
	unsigned int length = 3;

	float *d_B;
  float *d_C;
  float *d_A;
  cudaMalloc((void **)&d_B, length * length * sizeof(float));
  cudaMalloc((void **)&d_C, length * sizeof(float));
  cudaMalloc((void **)&d_A, length * sizeof(float));

  cudaMemcpy(d_B, h_B, length * length * sizeof(float), cudaMemcpyHostToDevice);
  cudaMemcpy(d_C, h_C, length * sizeof(float), cudaMemcpyHostToDevice);
    
  multMatrixVector<<<1, 32>>>(d_B, d_C, d_A, length);
    
  float *h_A = (float*) malloc (length*sizeof(float));
  cudaMemcpy(h_A, d_A, length * sizeof(float), cudaMemcpyDeviceToHost);
  
  for(int i=0; i<length; i++)
  {
    printf("%f, ", h_A[i]);
  }

  free(h_A);
  cudaFree(d_A);
  cudaFree(d_B);
  cudaFree(d_C);

	return 0;
}

Overwriting MMul/mmul.cu


### Compile

In [ ]:
!nvcc "MMul/mmul.cu" -o "MMul/a.out" $MAXWELL_ARCH_CMD

### Run

In [10]:
!"MMul/a.out"

8.000000, 26.000000, 44.000000, 


### Verify

In [11]:
!compute-sanitizer --tool memcheck "MMul/a.out"

========= COMPUTE-SANITIZER
8.000000, 26.000000, 44.000000, ========= ERROR SUMMARY: 0 errors


## Matrix Addition

In [12]:
%%writefile "MAdd/madd.cu"

#include <stdio.h>
#include <cuda.h>

#define TILE 16


// 1 thread 1 element
__global__ void addMMv1(float* B, float* C, float* A, unsigned int size) {
  int i = blockDim.x*blockIdx.x + threadIdx.x;

  if (i < size)
	{
		C[i] = A[i] + B[i];
	}
}

// 1 thread 1 col
__global__ void addMMv2(float* B, float* C, float* A, unsigned int size) {
  int y = blockDim.x*blockIdx.x + threadIdx.x;

  if (y < size)
	{
		for (int row_idx = 0; row_idx < size; ++row_idx)
		{
			int idx = row_idx * size + y;
			C[idx] = A[idx] + B[idx];
		}
	}

}

// 1 thread 1 row
__global__ void addMMv3(float* B, float* C, float* A, unsigned int size) {
  int x = blockDim.x*blockIdx.x + threadIdx.x;
  
  if (x < size)
	{
		int row_idx = x * size; 

		for (int col_idx = 0; col_idx < size; ++col_idx)
		{
			int idx = row_idx + col_idx;
			C[idx] = A[idx] + B[idx];
		}
	}
}
//

void generateRandomFlattenMatrix(float *M, unsigned int size) {
	for (int i = 0; i < size; ++i) {
	   M[i] = (rand() % 20) + 50;
	}
}



int main(int argc, char **argv)
{
	
	unsigned int length = 32;
  float *h_A = (float *) malloc(length*length);
  float *h_B = (float *) malloc(length*length);
	generateRandomFlattenMatrix(h_A, length);
	generateRandomFlattenMatrix(h_B, length);
  
  float *d_B;
  float *d_C;
  float *d_A;
  cudaMalloc((void **)&d_B, length * length * sizeof(float));
  cudaMalloc((void **)&d_C, length * length * sizeof(float));
  cudaMalloc((void **)&d_A, length * length * sizeof(float));

  cudaMemcpy(d_B, h_B, length * length * sizeof(float), cudaMemcpyHostToDevice);
  cudaMemcpy(d_A, h_A, length * length * sizeof(float), cudaMemcpyHostToDevice);

  // Uncomment the chosen one of the three below lines to get results form particular function
  //addMMv1<<<1, 64>>>(d_B, d_C, d_A, length);
  //addMMv2<<<1, 32>>>(d_B, d_C, d_A, length);
  addMMv3<<<1, 32>>>(d_B, d_C, d_A, length);
    
  float *h_C = (float*) malloc (length* length*sizeof(float));
  cudaMemcpy(h_C, d_C, length * length * sizeof(float), cudaMemcpyDeviceToHost);
  
  for(int i=0; i<length; i++)
  {
    printf("%f, ", h_C[i]);
  }

  free(h_C);
  cudaFree(d_A);
  cudaFree(d_B);
  cudaFree(d_C);

	return 0;
}

Overwriting MAdd/madd.cu


### Compile

In [ ]:
!nvcc "MAdd/madd.cu" -o "MAdd/a.out" $MAXWELL_ARCH_CMD

### Run

In [14]:
!"MAdd/a.out"

112.000000, 125.000000, 123.000000, 112.000000, 116.000000, 123.000000, 133.000000, 132.000000, 105.000000, 115.000000, 107.000000, 118.000000, 114.000000, 111.000000, 102.000000, 122.000000, 128.000000, 110.000000, 114.000000, 120.000000, 113.000000, 121.000000, 119.000000, 132.000000, 115.000000, 103.000000, 110.000000, 134.000000, 134.000000, 130.000000, 117.000000, 108.000000, 


### Verify

In [15]:
!compute-sanitizer --tool memcheck "MAdd/a.out"

========= COMPUTE-SANITIZER
112.000000, 125.000000, 123.000000, 112.000000, 116.000000, 123.000000, 133.000000, 132.000000, 105.000000, 115.000000, 107.000000, 118.000000, 114.000000, 111.000000, 102.000000, 122.000000, 128.000000, 110.000000, 114.000000, 120.000000, 113.000000, 121.000000, 119.000000, 132.000000, 115.000000, 103.000000, 110.000000, 134.000000, 134.000000, 130.000000, 117.000000, 108.000000, ========= ERROR SUMMARY: 0 errors


### Run profiler

In [16]:
!nvprof "MAdd/a.out"

112.000000, 125.000000, 123.000000, 112.000000, 116.000000, 123.000000, 133.000000, 132.000000, 105.000000, 115.000000, 107.000000, 118.000000, 114.000000, 111.000000, 102.000000, 122.000000, 128.000000, 110.000000, 114.000000, 120.000000, 113.000000, 121.000000, 119.000000, 132.000000, 115.000000, 103.000000, 110.000000, 134.000000, 134.000000, 130.000000, 117.000000, 108.000000, 


==8944== NVPROF is profiling process 8944, command: MAdd/a.out
==8944== Profiling application: MAdd/a.out
==8944== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   81.20%  15.616us         1  15.616us  15.616us  15.616us  addMMv3(float*, float*, float*, unsigned int)
                   11.15%  2.1440us         2  1.0720us  1.0560us  1.0880us  [CUDA memcpy HtoD]
                    7.65%  1.4720us         1  1.4720us  1.4720us  1.4720us  [CUDA memcpy DtoH]
      API calls:   74.90%  58.664ms         3  19.555ms  2.3000us  58.658ms  cudaMalloc
                   24.01%  18.806ms         1  18.806ms  18.806ms  18.806ms  cuDevicePrimaryCtxRelease
                    0.42%  331.70us         3  110.57us  54.600us  195.40us  cudaMemcpy
                    0.39%  305.40us         1  305.40us  305.40us  305.40us  cudaLaunchKernel
                    0.19%  149.30us         3  49.766us  2.3000us  141.30us  cudaFree
           